# Segmentation Notebook

### <font color='red'> After clicking on a code cell, press "Shift+Enter" to run the code, or click on the "Run" button in the toolbar above.<br>

### Replace "..." signs with the appropriate path to your data.
</font>

In [1]:
from tapenade.preprocessing import (
    local_image_equalization,
    normalize_intensity,
)
from tapenade import segmentation
import numpy as np
import tifffile
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.measure import regionprops

In [4]:
path = ...
array = tifffile.imread(Path(path) / "image.tif")
data_to_segment = array[:, 0, :, :]
stardist_model = Path(path) / "tapenade-stardist"

In [16]:
data_normalized = local_image_equalization(image=data_to_segment,box_size= 25, perc_low=1,perc_high=99)

In [ ]:
#if napari installed, you can check here the result of the normalization. In case of artifacts on the side of the image, you can add your mask to the function local_image_equalization
# import napari
# viewer=napari.Viewer()
# viewer.add_image(data_normalized,colormap='inferno')
# viewer.add_image(data_to_segment,colormap='inferno')
# napari.run()

In [ ]:
output = segmentation.predict_stardist(
    data_normalized,
    model_path=stardist_model,
    input_voxelsize=[1,0.6,0.6],
    normalize_input=False,
)

In [18]:
tifffile.imwrite(Path(path) / "segmentation.tif",output)

In [27]:
mask = tifffile.imread(Path(path) / "mask.tif")
output[mask==0]=0
tifffile.imwrite(Path(path) / "segmentation_masked.tif",output)


Threshold and normalize your division channel to run the prediction

In [ ]:

path_mask = ...
path_array = ...
path_output= ...
num= 1

mask = tifffile.imread(path_mask)
image = tifffile.imread(path_array)
division_channel = ...
dapi = ...

ph3 = normalize_intensity(
    image=division_channel,
    reference_image=dapi,
    mask=mask
)

blurred_data=ndi.gaussian_filter(ph3,sigma=1.2)

threshold = 0.8
blurred_data[blurred_data<threshold]=0

fig, (ax1,ax2) = plt.subplots(1,2, figsize=(12,5))
z =int(len(blurred_data)/2)
ax1.imshow(ph3[z,:,:])
ax2.imshow(blurred_data[z,:,:])
plt.tight_layout()

In [ ]:
stardist_model_divisions = ...
labels_division = segmentation.predict_stardist(blurred_data, model_path=stardist_model_divisions, input_voxelsize=[1, 1, 1], normalize_input=True)
print('Number of labels found :', len(np.unique(labels_division)))
tifffile.imsave(path_output,labels_division)

In [ ]:
#if napari installed
# import napari
# viewer=napari.Viewer()
# viewer.add_image(ph3,colormap='inferno')
# viewer.add_image(blurred_data,colormap='inferno')
# napari.run()

Apply mask to remove volumes that are out of the samples

In [ ]:
path_mask = ...
mask = tifffile.imread(path_mask)
labels_division=labels_division*mask.astype(bool)

Filter wrong volumes in the segmentation

In [ ]:
props=regionprops(labels_division)
histo=[]
for prop in props :
    bb = prop.slice #bounding box around the cell
    cell = prop.image #boolean array giving where the cell is
    histo.append(np.sum(cell))


plt.hist(histo,bins=100)
plt.title('Histogram of cell volumes')

In [ ]:
#choose the size to filter below
size_min =1000

labels_division_filtered = labels_division.copy()
unique_labels, label_counts = np.unique(labels_division, return_counts=True)

smallest_labels = unique_labels[np.argsort(label_counts)]
smallest_volumes = np.sort(label_counts)

for label, volume in zip(smallest_labels, smallest_volumes):
    if volume<size_min :
        labels_division_filtered[labels_division==label]=0

print('The prediction returned',len(np.unique(labels_division)),'labels, after filtering we have now',len(np.unique(labels_division_filtered)),'labels')

In [ ]:
# viewer=napari.Viewer()
# viewer.add_image(ph3,scale=scale)
# viewer.add_labels(labels_division_filtered,scale=scale,name='labels_filtered')
# viewer.add_labels(labels_division,scale=scale,name='labels_not_filtered')
# napari.run()